By implementing a baseline recommandation we will be able to compare the results of our algorithms. Our comparison algorithm will recommend a random product among the top 10 best-selling products in the store.

As *Google Collab®* doen't has Spark installed by default, next block prepare the environment to can work with Spark. Also as this platform is efimer the data is managed with a mounted volume to *Google Drive®*. 

In [ ]:
!apt-get update

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa

In [ ]:
try:
  # Only it runs in google collab
  from google.colab import drive # if this package don't exist we aren't con google colab
  import os
  # Install spark and its requirements
  !apt-get install openjdk-8-jdk-headless -qq > /dev/null
  !wget -nc -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
  !tar xf spark-3.2.0-bin-hadoop3.2.tgz
  !pip install -q pyspark
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"
  drive.mount('/content/drive')
  base_path = '/content/drive/MyDrive/data/' # Google Drive
except:
  # It runs in local 
  base_path = ''


     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 198 kB 43.9 MB/s 
Mounted at /content/drive


Importing libraries to work with Spark and initializate a SparkSession.

In [ ]:
# Import Spark libs
from pyspark.sql import SparkSession
from IPython.display import display, HTML
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col


# Initiate SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.sql.execution.arrow.enabled", "true") \
    .getOrCreate()

The path of files. This uses a base path in order to be able to work this Jupyter notebook wherever (at moment in colab and local).

In [ ]:
# File Path
file_products = base_path + 'ARTICLES.csv'
file_families = base_path + 'AECOC.csv'
file_sells = base_path + 's20.csv'

#### Products file

In [ ]:
# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ";"

df_product = spark.read.format('csv') \
  .option("charset", "ISO-8859-1") \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_products)
df_product.limit(5).toPandas()

,ARTICLE,DESCRIPCIO,SECTOR,SECCIO,FAMILIA,DESC
0,10000,LIMPIACRISTALES CON,3,1,6,03*01*06
1,10001,LIMPIACRISTALES RECA,3,1,6,03*01*06
2,10002,LIMPIACRISTALES BONA,3,1,6,03*01*06
3,10003,LIMPIACRISTALES BONA,3,1,6,03*01*06
4,10004,FREGASUELOS CAG 1 L,3,1,6,03*01*06


It make a projection of this file with selected columns and normalized its name.

In [ ]:
print ("total products: ", df_product.count())
df_product = df_product.selectExpr('ARTICLE as product_id','DESC as family_id', 'DESCRIPCIO as product_desc' )
df_product.limit(5).toPandas()


total products:  72200


,product_id,family_id,product_desc
0,10000,03*01*06,LIMPIACRISTALES CON
1,10001,03*01*06,LIMPIACRISTALES RECA
2,10002,03*01*06,LIMPIACRISTALES BONA
3,10003,03*01*06,LIMPIACRISTALES BONA
4,10004,03*01*06,FREGASUELOS CAG 1 L


#### Families file

In [ ]:
# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ";"

df_families = spark.read.format('csv') \
  .option("charset", "ISO-8859-1") \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_families)

df_families.limit(5).toPandas()

,FAMILIA,DESCRIPCIO.SECTOR,DESCRIPCIO.SECCIO,DESC.FAMILIA,_c4,_c5,_c6,_c7
0,00*00*00,Desconocido,,Desconeguda,None,None,None,None
1,01*01*01,Alimentacion y Bebidas,ALIMENTACIÓN SECA,Aceites,None,None,None,None
2,01*01*02,Alimentacion y Bebidas,ALIMENTACIÓN SECA,Cafés y sucedáneos,None,None,None,None
3,01*01*03,Alimentacion y Bebidas,ALIMENTACIÓN SECA,Infusiones,None,None,None,None
4,01*01*04,Alimentacion y Bebidas,ALIMENTACIÓN SECA,Chocolates,None,None,None,None


In [ ]:
schema = StructType([
    StructField("family_id", StringType(), True),
    StructField("sector_desc", StringType(), True),
    StructField("section_desc", StringType(), True),
    StructField("family_desc", StringType(), True),
    StructField("not_use1", StringType(), True),
    StructField("not_use2", StringType(), True),
    StructField("not_use3", StringType(), True),
    StructField("not_use4", StringType(), True)
    ])

df_families = spark.read.format('csv') \
  .schema(schema) \
  .option("charset", "ISO-8859-1") \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_families)


print ("total families: ", df_families.count())

df_families.select(col('family_id'), col('family_desc') ).limit(5).toPandas()


total families:  816


,family_id,family_desc
0,00*00*00,Desconeguda
1,01*01*01,Aceites
2,01*01*02,Cafés y sucedáneos
3,01*01*03,Infusiones
4,01*01*04,Chocolates


In [ ]:
# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ";"

schema = StructType([
    StructField("invoice_id", StringType(), True),
    StructField("product_id", IntegerType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("amount", IntegerType(), True),
    StructField("checkout", IntegerType(), True),
    StructField("date", IntegerType(), True),
    StructField("hour", IntegerType(), True),])

df_sells = spark.read.format('csv') \
  .schema(schema) \
  .option("charset", "ISO-8859-1") \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_sells)

df_sells.limit(5).toPandas()


,invoice_id,product_id,quantity,amount,checkout,date,hour
0,2027-T0132C01-000001,6252,4000000,419,1,18995,31737
1,2027-T0132C01-000001,4465,1000000,148,1,18995,31737
2,2027-T0132C01-000001,4465,1000000,132,1,18995,31737
3,2027-T0132C01-000001,4465,1000000,131,1,18995,31737
4,2027-T0132C01-000001,4465,1000000,130,1,18995,31737


In [ ]:
from datetime import datetime,timedelta

print("total sells: ", df_sells.count())

StartDate = "12/31/1967"
Date = datetime.strptime(StartDate, "%m/%d/%Y")

df_sells = df_sells.rdd \
  .map(lambda x: (x[0],x[1],x[2]/1000000,x[3]/100,x[4],Date.replace(hour=0, minute=0, second=0, microsecond =0)+timedelta(days=x[5], seconds=x[6]))) \
  .toDF(['invoice_id','product_id', 'quantity','amount','checkout','datetime'])

df_sells.limit(5).toPandas()

total sells:  1156642


,invoice_id,product_id,quantity,amount,checkout,datetime
0,2027-T0132C01-000001,6252,4.0,4.19,1,2020-01-02 08:48:57
1,2027-T0132C01-000001,4465,1.0,1.48,1,2020-01-02 08:48:57
2,2027-T0132C01-000001,4465,1.0,1.32,1,2020-01-02 08:48:57
3,2027-T0132C01-000001,4465,1.0,1.31,1,2020-01-02 08:48:57
4,2027-T0132C01-000001,4465,1.0,1.30,1,2020-01-02 08:48:57


In [ ]:
products_to_avoid = [
                     8055, # plastic bag
                     9117, # plastic bag
                     8419, # raffia bag
                     ]
df_sells = df_sells[~df_sells['product_id'].isin(products_to_avoid)]

#Determine the top 10 best-selling

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.functions import regexp_replace
from pyspark.sql.window import Window

In [ ]:
sells_2020_byproduct = df_sells.groupBy("product_id").sum("quantity")
sells_2020_byproduct_wdesc = sells_2020_byproduct.join(df_product, on='product_id')
sells_2020_byproduct_wdesc = sells_2020_byproduct_wdesc.sort(F.col('sum(quantity)').desc())
sells_2020_byproduct_top10 = sells_2020_byproduct_wdesc.limit(10)
sells_2020_byproduct_top10.show()

+----------+-------------+---------+--------------------+
|product_id|sum(quantity)|family_id|        product_desc|
+----------+-------------+---------+--------------------+
|      7550|      20488.0| 01*03*01|LECHE ENTERA BONAREA|
|      7551|      15087.0| 01*03*01| LECHE SEMIDESNATADA|
|      7665|      14994.0| 01*04*01|AGUA BONï¿½REA 1,...|
|      6253|      14086.0| 02*01*07|HUEVOS M RUBIO BONAR|
|      6252|      11105.0| 02*01*07|HUEVOS L RUBIO BONAR|
|      7552|       9068.0| 01*03*01|LECHE DESNATADA BONA|
|      2111|       8574.0| 02*06*01|PLATANOS CANARIAS BO|
|      6255|       6727.0| 02*01*07|HUEVOS DE CORRAL RUB|
|      8127|       6642.0| 01*04*04|CERVEZA BONï¿½REA...|
|      3060|       5812.0| 02*05*01|YOGUR NATURAL BONARE|
+----------+-------------+---------+--------------------+



In [ ]:
sells_2020_byproduct_id_top10 = sells_2020_byproduct_top10[["product_id"]]
sells_2020_byproduct_id_top10 = sells_2020_byproduct_id_top10.withColumnRenamed("product_id", "recommendation")

#Apply recommendation

In [ ]:
df_sells_by_invoice_id = df_sells.groupBy("invoice_id", "datetime").agg(F.collect_list("product_id").alias("product_id"), F.collect_list("quantity").alias("quantities"), F.collect_list("amount").alias("amounts"))
df_sells_by_invoice_id = df_sells_by_invoice_id.filter(F.size(F.col("product_id")) > 2)

In [ ]:
sells_test_results = df_sells_by_invoice_id.withColumn('product_expected', F.col('product_id')[0])
sells_test_results = sells_test_results.withColumn('product_without_expected', F.expr("slice(product_id, 2, SIZE(product_id))"))
sells_test_results = sells_test_results.withColumn('quantities_without_expected', F.expr("slice(quantities, 2, SIZE(quantities))"))
sells_test_results = sells_test_results.withColumn('amounts_without_expected', F.expr("slice(amounts, 2, SIZE(amounts))"))
sells_test_results = sells_test_results.withColumn('invoice_id', regexp_replace('invoice_id', '2027-T0132C01-', ''))
sells_test_results = sells_test_results.withColumn('invoice_id', sells_test_results["invoice_id"].cast(IntegerType()))
#sells_test_results.show()

In [ ]:
dfB = sells_2020_byproduct_id_top10.withColumn("index", F.row_number().over(Window.orderBy("recommendation")) - 1)
dfA = sells_test_results.withColumn("index", (F.rand() * dfB.count()).cast("bigint"))

sells_test_results_w_rec = dfA.join(dfB, on="index", how="left").drop("index")
#sells_test_results_w_rec.show()

#Compare predictions

In [ ]:
sells_test_results_w_recommendation = sells_test_results_w_rec.withColumn('success_recomended', F.col('recommendation') == F.col('product_expected'))
#sells_test_results_w_recommendation.show()

In [ ]:
ok = sells_test_results_w_recommendation.filter(F.col('success_recomended') == 'true')
ok.show()

+----------+-------------------+--------------------+--------------------+--------------------+----------------+------------------------+---------------------------+------------------------+--------------+------------------+
|invoice_id|           datetime|          product_id|          quantities|             amounts|product_expected|product_without_expected|quantities_without_expected|amounts_without_expected|recommendation|success_recomended|
+----------+-------------------+--------------------+--------------------+--------------------+----------------+------------------------+---------------------------+------------------------+--------------+------------------+
|       206|2020-01-02 13:16:45|[7551, 3046, 7696...|[6.0, 1.0, 1.0, 1...|[3.35, 1.29, 0.9,...|            7551|    [3046, 7696, 6473...|       [1.0, 1.0, 1.0, 1...|    [1.29, 0.9, 2.53,...|          7551|              true|
|      1159|2020-01-04 18:27:19|[2111, 4490, 4355...|[1.0, 1.0, 1.0, 1...|[1.72, 1.88, 2.05...|     

In [ ]:
nb_ok = ok.count()
nb_ok

1009

In [ ]:
nb_ok = ok.count()
total = sells_test_results_w_recommendation.count()
percent = 100 * nb_ok / total
percent

0.8461074028108543